In [ ]:
import mediapipe as mp
from   sympy import re 

import cv2

import math
import numpy 


from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(
    IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))
print(volume.GetMute())
print(volume.GetMasterVolumeLevel())
print(volume.GetVolumeRange())
volume.SetMasterVolumeLevel(-20.0, None)









mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()



cap = cv2.VideoCapture(0)

while True:
    success, image = cap.read()
    image = cv2.cvtColor(image , cv2.COLOR_BGR2RGB)
    results = hands.process(image)
    
    if results.multi_hand_landmarks:
        for handlms in results.multi_hand_landmarks:
            lmlist = []
            for id ,lm in enumerate (handlms.landmark):
                # print(id , lm )
                h, w , c = image.shape
                cx , cy = int(lm.x*w) , int(lm.y*h)
                # print(id , cx, cy)
                lmlist.append( [id , cx , cy])
                # print(lmlist)
            if lmlist:
                x1 , y1 = lmlist[4][1] , lmlist[4][2]
                x2 , y2 = lmlist[8][1] , lmlist[8][2]
                cv2.circle (image ,( x1 , y1 ) , 13 , ( 167 , 21 , 56 ) ,cv2.FILLED )
                cv2.circle (image ,( x2 , y2 ) , 13 ,( 167 , 21 , 56 ),cv2.FILLED)
                cv2.line( image , (x1 , y1 ) , (x2, y2 ) , (2 , 45 , 433) , 5 )
                length = math.hypot(x2- x1 , y2 - y1)
                # print(length)
                if length< 50 :
                    z1 = ( x1 + x2 )//2
                    z2 = ( y1 + y2)//2 
                    cv2.circle(image , (z1, z2) ,25 , ( 2 , 123 , 543), cv2.FILLED )
                    
            volRange =   volume.GetVolumeRange()
            minVol = volRange[0]
            maxVol = volRange[1]
            vol = numpy.interp(length, [50 , 300] , [minVol , maxVol])
            volBar = numpy.interp(length , [50 , 300] , [400 , 150])
            volPer = numpy.interp(length , [50, 300] , [0 , 100])
        
            volume.SetMasterVolumeLevel( vol , None )   
            
            cv2.rectangle(image , str(int(volPer)) , (40 , 450) , cv2.FONT_HERSHEY_PLAIN , 4 (24, 34,34) ,3 )  
                    
        
    cv2.imshow( "Hand" , image )
    cv2.waitKey(3)



